# Preguntas y Respuestas con RAG Langchain

Usando Chroma DB and LangChain para contestar preguntas de un texto, con una base de datos local. Guardamos los embaddings, y los usamos después.

In [9]:
!pip install chromadb 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.4 MB/s eta 0:00:00ta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 2.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 9.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.2/197.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.5/528.5 kB 11.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 19.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━

In [1]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings

In [3]:
from dotenv import load_dotenv
import os
# Initialize the client
load_dotenv()

secret_string = os.getenv("OPENAI_API_KEY")


## Cargar y procesar documentos.
aqui cargamos los documentos sobre los que vamosa  aplicar el Q&A

Los dividimos en trozos (chunks). ASi buscamos los más relevantes y los pasamos al LLM.

In [6]:
# Load and process the text
loader = TextLoader('kennedy.txt')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

## Inicializamos PeristedChromaDB

Creamos los embeddings. Y los guardamos en la db

In [10]:

# Directorio donde se guardará la db
persist_directory = 'db'

engine = "gpt-4"
embedding = OpenAIEmbeddings(api_key=secret_string, model="text-embedding-3-large")

vectordb = Chroma.from_documents(documents=texts, embedding=embedding, persist_directory=persist_directory)

## Persist the Database
In a notebook, we should call `persist()` to ensure the embeddings are written to disk.
This isn't necessary in a script - the database will be automatically persisted when the client object is destroyed.

In [11]:
vectordb.persist()
vectordb = None

## Cargamos la db del disco
`persist_directory` es el directorio donde se va a guardar la db y `embedding_function` tiene que ser la misma que cunado instanciamos a db. Se encargara de las respuestas.

In [13]:
# Cargamos la base de datos persistente del disco y usamos un langchain retrieval
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(api_key=secret_string),
    retriever=vectordb.as_retriever(
        search_kwargs={"k": 5})
    )

## Preguntas y respuestas

Con langchainy RAG

In [14]:
query = "why did we go to the moon"
result = qa.invoke(query)
result.get("result")

Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


' We chose to go to the moon in order to push the limits of human knowledge and progress, and to serve as a challenging goal that would bring out the best of our abilities and skills. This decision was made despite the inherent risks and difficulties involved.'

## Limpieza

Cuando hayas terminado con la base de datos, puedes eliminarla del disco. Puedes eliminar la colección específica con la que está trabajando (si tiene varias) o eliminar toda la base de datos destruyendo el directorio de persistencia.

In [ ]:
# limpiamos la colección
vectordb.delete_collection()
vectordb.persist()

# nos cargamos el directorio de persistencia
!rm -rf db/

Persisting DB to disk, putting it in the save folder db
